In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import time
from urllib.parse import urlparse, parse_qs
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
import sqlite3
import os

In [3]:
conn = sqlite3.connect('C:/Users/ITSC/Desktop/Project/MiniProject1/noodle_street/web/db.sqlite3')
cur = conn.cursor()

In [6]:
example_data = [
    ('john_doe', 'John', 'Doe', 'john@example.com', 'password123', 0, 1, 0, None, date.today(), '1990-01-01', 'users/profile/default.jpg'),
    ('jane_smith', 'Jane', 'Smith', 'jane@example.com', 'password456', 0, 1, 0, None, date.today(), '1985-05-15', 'users/profile/default.jpg')
]

In [11]:
insert_query = """
INSERT INTO users_user (username, first_name, last_name, email, password, is_staff, is_active, is_superuser, last_login, date_joined, birthday, profile_image)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
"""

In [90]:
# 영업시간 가져오는 함수
def hours_raw(url):
    
    soup = BeautifulSoup(driver.page_source, "lxml")

    busihours = soup.select("div.busi-hours ul.list")[0].text.strip().split("\n")
    busitoday = soup.select("div.busi-hours-today ul.list")[0].text.strip().split("\n")
    
    
    hours = [i.strip() for i in busihours + busitoday if i != ""]
    
    weekdays = ["월", "화", "수", "목", "금", "토", "일"]
    
    busi_hours = []
    
    for i in range(len(hours)):
        if any(char in hours[i] for char in weekdays) and hours[i] != "휴무일":
            busi_hours.append(hours[i])

            try:
                if "확인 필요" in hours[i+1]:
                    busi_hours.append(hours[i+1])

                elif "휴무일" in hours[i+1]:
                    busi_hours.append(hours[i+1])

                elif "영업시간" in hours[i+1]:
                    busi_hours.append(hours[i+1])

            except IndexError:
                busi_hours.append("")                    
                    

            try:
                if "브레이크타임" in hours[i+2]:
                    busi_hours.append(hours[i+2])

                    
                    try:
                        if "라스트오더" in hours[i+3]:
                            busi_hours.append(hours[i+3])

                        else:
                            busi_hours.append("") 

                    except IndexError:
                        busi_hours.append("")
                    
                elif "라스트오더" in hours[i+2]:
                    busi_hours.append("")
                    busi_hours.append(hours[i+2])

                else:
                    busi_hours.append("")
                    busi_hours.append("")

            except IndexError:
                busi_hours.append("")    
                busi_hours.append("")

    return busi_hours

In [154]:
# 나머지 가게 상세정보들 찾는 함수
def get_restaurant_info(driver, link):
    # 요소를 다시 찾기 위해 페이지를 다시로드
    driver.get(link)
    driver.implicitly_wait(3)

    # 영업시간 더보기 누르기
    plus = driver.find_element(By.CSS_SELECTOR, "a.more-btn")
    plus.click()
    driver.implicitly_wait(3)

    # 가게 정보를 담을 리스트 초기화
    restaurant_info = []

    # 현재 상세정보 가게 url
    now_url = driver.current_url
    restaurant_info.append(now_url)
    
    # 가게이름 찾기
    name = driver.find_element(By.CSS_SELECTOR, "h1.tit")
    restaurant_info.append(name.text)

    # 평점 찾기
    try:
        rating = driver.find_element(By.CSS_SELECTOR, "span.point")
        restaurant_info.append(rating.text)
    except NoSuchElementException:
        restaurant_info.append("")

    # 주소찾기(주소가 서울시00구00역과 숫자로 나뉘어 져있어 합침)
    addr1 = driver.find_elements(By.CSS_SELECTOR, "li.locat > a")
    addr2 = driver.find_element(By.CSS_SELECTOR, "li.locat > span").text
    addr3 = driver.find_elements(By.CSS_SELECTOR, "li.locat > a")[1]
    restaurant_info.append(" ".join([addr.text for addr in addr1]) + addr2)
    restaurant_info.append(addr3.text)

    # 가게 전화번호 찾기
    try:
        tel = driver.find_element(By.CSS_SELECTOR, "li.tel")
        restaurant_info.append(tel.text)
    except NoSuchElementException:
        restaurant_info.append("")

    # 주차가능 여부
    try:
        parking = driver.find_element(By.CSS_SELECTOR, "li.char")
        restaurant_info.append(parking.text)
    except NoSuchElementException:
        restaurant_info.append("")

    # 영업시간 가져오기
    restaurant_info.extend(hours_raw(link))

    # 이미지 url 찾기
    try:
        img = driver.find_element(By.CSS_SELECTOR, "div.btn-gallery-open>img")
        restaurant_info.append(img.get_attribute("src"))
    except NoSuchElementException:
        restaurant_info.append("")

    return restaurant_info

In [155]:
driver = webdriver.Chrome()
driver.implicitly_wait(5)
url = "https://www.diningcode.com/list.dc?query=%EC%84%9C%EC%9A%B8%20%EB%A9%B4%EC%9A%94%EB%A6%AC&addr=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%B6%81%EA%B5%AC%20%2B%EC%88%98%EC%9C%A0%EC%8B%9C%EC%9E%A5"
driver.get(url)

# 로딩 대기 시간을 설정 
driver.implicitly_wait(3)

data = driver.find_element(By.ID, "arrow")
data.click()
driver.implicitly_wait(3)

# end 키 6번 클릭 후 1.5초 기다림
for i in range(6):
    data.send_keys(Keys.END)
    time.sleep(1.5)

# 'a.sc-ilxdoh' 요소 찾아서 리스트에 담기
e = driver.find_elements(By.XPATH, '//div[@class="sc-gYrpUN eJNRnf Poi__List__Wrap"]/a')
link_list = [i.get_attribute('href') for i in e]

# 각 가게 정보를 담을 리스트 초기화
restaurant_info_list = []

# 결과 출력
for i, link in enumerate(link_list):
    restaurant_info = get_restaurant_info(driver, link)
    restaurant_info_list.append(restaurant_info)

# 각 가게 정보를 출력
for i, restaurant_info in enumerate(restaurant_info_list):
    print(f"가게 {i + 1} 정보:")
    print(restaurant_info)
    print("\n")

가게 1 정보:
['https://www.diningcode.com/profile.php?rid=iPzsbyCGBKoM', '항아리엄마손팥해물칼국수', '4 (1명의 평가)', '서울특별시 강북구 미아동217-1', '강북구', '02-988-7979', '노포, 좌식테이블', '02.17 (토)', '확인 필요', '', '', '02.18 (일)', '확인 필요', '', '', '02.19 (월)', '확인 필요', '', '', '02.20 (화)', '확인 필요', '', '', '02.21 (수)', '확인 필요', '', '', '02.22 (목)', '확인 필요', '', '', '[오늘] 02.16 (금)', '영업시간 확인 필요', '', '', 'https://d12zq4w4guyljn.cloudfront.net/750_750_20220117035017_photo1_f7c1425b9393.jpg']


가게 2 정보:
['https://www.diningcode.com/profile.php?rid=8wcmPyD61Rnq', '생국수 본가', '3 (1명의 평가)', '서울 강북구 수유동54-5 수유전통시장', '강북구', '', '저렴, 점심식사, 저녁식사, 제로페이', '02.17 (토)', '영업시간: 11:00 - 20:00', '', '', '02.18 (일)', '영업시간: 11:00 - 20:00', '', '', '02.19 (월)', '영업시간: 11:00 - 20:00', '', '', '02.20 (화)', '영업시간: 11:00 - 20:00', '', '', '02.21 (수)', '영업시간: 11:00 - 20:00', '', '', '02.22 (목)', '영업시간: 11:00 - 20:00', '', '', '[오늘] 02.16 (금)', '영업시간: 11:00 - 20:00', '', '', 'https://d12zq4w4guyljn.cloudfront.net/750_750_20201119094214_photo1

In [165]:
def weekly_hours(restaurant_info):
    
    weekday_data = [i for i in restaurant_info[7:35] if restaurant_info.index(i)%4 == 3]
    hours_data = ["" for _ in range(35)]
    
    for i in range(0,7):
        if "월" in weekday_data[i]:
            mon_hours = daily_hours(i*4)
            
            hours_data[0] = mon_hours[0] # mon_start 값 지정
            hours_data[1] = mon_hours[1] # mon_end 값 지정
            hours_data[2] = mon_hours[2] # mon_break_start 값 지정
            hours_data[3] = mon_hours[3] # mon_break_end 값 지정
            hours_data[4] = mon_hours[4] # mon_last_order 값 지정

        elif "화" in weekday_data[i]:
            tue_hours = daily_hours(i*4)
            
            hours_data[5] = tue_hours[0] # tue_start 값 지정
            hours_data[6] = tue_hours[1] # tue_end 값 지정
            hours_data[7] = tue_hours[2] # tue_break_start 값 지정
            hours_data[8] = tue_hours[3] # tue_break_end 값 지정
            hours_data[9] = tue_hours[4] # tue_last_order 값 지정

        elif "수" in weekday_data[i]:
            wed_hours = daily_hours(i*4)
            
            hours_data[10] = wed_hours[0] # wed_start 값 지정
            hours_data[11] = wed_hours[1] # wed_end 값 지정
            hours_data[12] = wed_hours[2] # wed_break_start 값 지정
            hours_data[13] = wed_hours[3] # wed_break_end 값 지정
            hours_data[14] = wed_hours[4] # wed_last_order 값 지정

        elif "목" in weekday_data[i]:
            thu_hours = daily_hours(i*4)
            
            hours_data[15] = thu_hours[0] # thu_start 값 지정
            hours_data[16] = thu_hours[1] # thu_end 값 지정
            hours_data[17] = thu_hours[2] # thu_break_start 값 지정
            hours_data[18] = thu_hours[3] # thu_break_end 값 지정
            hours_data[19] = thu_hours[4] # thu_last_order 값 지정


        elif "금" in weekday_data[i]:
            fri_hours = daily_hours(i*4)
            
            hours_data[20] = fri_hours[0] # fri_start 값 지정
            hours_data[21] = fri_hours[1] # fri_end 값 지정
            hours_data[22] = fri_hours[2] # fri_break_start 값 지정
            hours_data[23] = fri_hours[3] # fri_break_end 값 지정
            hours_data[24] = fri_hours[4] # fri_last_order 값 지정


        elif "토" in weekday_data[i]:
            sat_hours = daily_hours(i*4)
            
            hours_data[25] = sat_hours[0] # sat_start 값 지정
            hours_data[26] = sat_hours[1] # sat_end 값 지정
            hours_data[27] = sat_hours[2] # sat_break_start 값 지정
            hours_data[28] = sat_hours[3] # sat_break_end 값 지정
            hours_data[29] = sat_hours[4] # sat_last_order 값 지정


        else:
            sun_hours = daily_hours(i*4)
            
            hours_data[30] = sun_hours[0] # sun_start 값 지정
            hours_data[31] = sun_hours[1] # sun_end 값 지정
            hours_data[32] = sun_hours[2] # sun_break_start 값 지정
            hours_data[33] = sun_hours[3] # sun_break_end 값 지정
            hours_data[34] = sun_hours[4] # sun_last_order 값 지정
            
    return hours_data


In [76]:
# 각 요일의 영업 시간, 브레이크타임, 라스트오더를 추출하는 함수

def daily_hours(i):
    my_str0 = restaurant_info[i + 8]
    
    working_hours = []
    
    if "영업시간:" in my_str0:
        working_hours.append(my_str0.replace("영업시간: ", "").split(" - ")[0])
        working_hours.append(my_str0.replace("영업시간: ", "").split(" - ")[1])

        my_str1 = restaurant_info[i + 9]
        if my_str1 == "":
            working_hours.append("")
            working_hours.append("")

        else:
            working_hours.append(my_str1.replace("브레이크타임: ", "").split(" - ")[0])
            working_hours.append(my_str1.replace("브레이크타임: ", "").split(" - ")[1])

        my_str2 = restaurant_info[i + 10]
        
        if my_str2 == "":
            working_hours.append("")

        else:
            working_hours.append(my_str2.replace("라스트오더: ", ""))

    elif "확인 필요" in my_str0:
        working_hours = ["확인 필요", "", "", "", ""]
            

    elif "휴무일" in my_str0:
        working_hours = ["휴무일", "", "", "", ""]      
            
    else:
        working_hours = ["", "", "", "", ""]
        
    return working_hours

In [189]:
# 음식점 정보를 db에 입력하는 함수 (커서 지정이 포함되어 있음!)

def restaurantinsert(restaurant_info):
    conn = sqlite3.connect('C:/Users/ITSC/Desktop/Project/MiniProject1/noodle_street/web/db.sqlite3')
    cur = conn.cursor()
    
    data = ["" for _ in range(44)]
    insert_query = """
    INSERT INTO single_pages_restraunt(
    'name', 'address', 'address_gu', 'phone_no', 'parking', 'rating', 'rating_count', 
    'mon_start', 'mon_end', 'mon_break_start', 'mon_break_end', 'mon_last_order',
    'tue_start', 'tue_end', 'tue_break_start', 'tue_break_end', 'tue_last_order',
    'wed_start', 'wed_end', 'wed_break_start', 'wed_break_end', 'wed_last_order',
    'thu_start', 'thu_end', 'thu_break_start', 'thu_break_end', 'thu_last_order',
    'fri_start', 'fri_end', 'fri_break_start', 'fri_break_end', 'fri_last_order',
    'sat_start', 'sat_end', 'sat_break_start', 'sat_break_end', 'sat_last_order',
    'sun_start', 'sun_end', 'sun_break_start', 'sun_break_end', 'sun_last_order', 
    'url', 'image'
    )
    VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """

    data[0] = restaurant_info[1] # name 값 지정
    data[1] = restaurant_info[3] # address 값 지정
    data[2] = restaurant_info[4] # address_gu 값 지정
    
    # phone_no 값 지정
    if restaurant_info[5] != "": 
        data[3] = restaurant_info[5]
        
    else:
        data[3] = ""

    # parking 값 지정
    if "주차" in restaurant_info[6]: 
        data[4] = 1

    else:
        data[4] = 0

    # rating, rating_count 값 지정    
    if restaurant_info[2] != "":
        data[5] = restaurant_info[2].split(" ")[0] # rating 값 지정
        data[6] = restaurant_info[2].split(" ")[1][1:].replace("명의", "") # rating_count 값 지정
        
    else:
        data[5] = ""
        data[6] = ""
    
    busi_hours = weekly_hours(restaurant_info) # 요일별 영업 시간 입력
    for i in range(len(busi_hours)):
        data[i+7] = busi_hours[i]
    
    data[42] = restaurant_info[0] # url 값 지정
    
    # image 값 지정
    if restaurant_info[35] != "":
        data[43] = restaurant_info[35] 
        
    else:
        data[43] = ""

    
    print(data)
    print(len(data))
    
    cur.execute(insert_query, data)
    
    cur.close()
    conn.commit()
    conn.close()

In [192]:
restaurant_info = restaurant_info_list[12]
print(restaurant_info)

['https://www.diningcode.com/profile.php?rid=mrSmu4earorw', '공릉동 닭한마리 수유점', '', '서울특별시 강북구 미아동187-40', '강북구', '0507-1422-3571', '따끈한국물, 좌식, 주차, 배달', '02.17 (토)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '02.18 (일)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '02.19 (월)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '02.20 (화)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '02.21 (수)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '02.22 (목)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '[오늘] 02.16 (금)', '영업시간: 11:00 - 23:00', '', '라스트오더: 22:00', '']


In [193]:
restaurantinsert(restaurant_info)

['공릉동 닭한마리 수유점', '서울특별시 강북구 미아동187-40', '강북구', '0507-1422-3571', 1, '', '', '11:00', '23:00', '', '', '22:00', '11:00', '23:00', '', '', '22:00', '11:00', '23:00', '', '', '22:00', '11:00', '23:00', '', '', '22:00', '11:00', '23:00', '', '', '22:00', '11:00', '23:00', '', '', '22:00', '11:00', '23:00', '', '', '22:00', 'https://www.diningcode.com/profile.php?rid=mrSmu4earorw', '']
44
